In [49]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.modules.loss as loss
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [0]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=4,
                          num_workers=2,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=4,
                          num_workers=2,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=4,
                          num_workers=2,
                          shuffle=True)

In [0]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        
epochs     = 20
#criterion = # Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = loss.CrossEntropyLoss()
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [0]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    computing_device = torch.device('cuda')
else:
    computing_device = torch.device('cuda')

def train():
    for epoch in range(epochs):
        ts = time.time()
        lossSum = 0
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()


            #inputs = X.to(computing_device)
            inputs = X.cuda()
            labels = Y.cuda()
            #labels = Y.to(computing_device)
            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            lossSum += loss.data
            loss.backward()
            optimizer.step()
            if iter > 1:
                break
            if iter % 10 == 0:
                None
                #print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
            #print(torch.cuda.memory_allocated(device=None))
        print("Finish epoch {}, time elapsed {}, loss {}".format(epoch, time.time() - ts, lossSum))
        torch.save(fcn_model, 'best_model')
        
        val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    vals = val_loader
    lossSum = 0
    for iter, (X, tar, Y) in enumerate(vals):
        with torch.no_grad():
            inputs = X.cuda()
            labels = Y.cuda()
#             print(torch.cuda.memory_allocated(device=None))
            outputs = fcn_model(inputs)
            lossSum += criterion(outputs, labels).data
        if iter > 1:
            break
    print("Epoch:", epoch, ", Loss: ", lossSum)
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    
def test():
    None
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    train()